## What可视化

2D Grand CAM可视化模块

In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import monai
from glob import glob
import matplotlib.pyplot as plt

mydir = r'E:\function\pm_data\skin4clf_out\images'
samples = [os.path.join(mydir, f) for f in os.listdir(mydir) if f.endswith('.jpg') or f.endswith('.png')]
samples

['E:\\function\\pm_data\\skin4clf_out\\images\\ISIC_0000021_downsampled.jpg',
 'E:\\function\\pm_data\\skin4clf_out\\images\\ISIC_0000012.jpg',
 'E:\\function\\pm_data\\skin4clf_out\\images\\ISIC_0000002.jpg',
 'E:\\function\\pm_data\\skin4clf_out\\images\\ISIC_0000018_downsampled.jpg',
 'E:\\function\\pm_data\\skin4clf_out\\images\\ISIC_0000015.jpg',
 'E:\\function\\pm_data\\skin4clf_out\\images\\ISIC_0000017_downsampled.jpg',
 'E:\\function\\pm_data\\skin4clf_out\\images\\ISIC_0000022_downsampled.jpg',
 'E:\\function\\pm_data\\skin4clf_out\\images\\ISIC_0000008.jpg',
 'E:\\function\\pm_data\\skin4clf_out\\images\\ISIC_0000013.jpg',
 'E:\\function\\pm_data\\skin4clf_out\\images\\ISIC_0000020_downsampled.jpg',
 'E:\\function\\pm_data\\skin4clf_out\\images\\ISIC_0000019_downsampled.jpg',
 'E:\\function\\pm_data\\skin4clf_out\\images\\ISIC_0000016.jpg',
 'E:\\function\\pm_data\\skin4clf_out\\images\\ISIC_0000007.jpg',
 'E:\\function\\pm_data\\skin4clf_out\\images\\ISIC_0000006.jpg',
 'E:

## 确定可视化模型

通过关键词获取要提取那一层进行可视化。

### 支持的模型名称

模型名称替换代码中的 `model_name`变量的值。

| **模型系列** | **模型名称**                                                 |
| ------------ | ------------------------------------------------------------ |
| AlexNet      | alexnet                                                      |
| VGG          | vgg11, vgg11_bn, vgg13, vgg13_bn, vgg16, vgg16_bn, vgg19_bn, vgg19 |
| ResNet       | resnet18, resnet34, resnet50, resnet101, resnet152, resnext50_32x4d, resnext101_32x8d, wide_resnet50_2, wide_resnet101_2 |
| DenseNet     | densenet121, densenet169, densenet201, densenet161           |
| Inception    | googlenet, inception_v3                                      |
| SqueezeNet   | squeezenet1_0, squeezenet1_1                                 |
| ShuffleNetV2 | shufflenet_v2_x2_0, shufflenet_v2_x0_5, shufflenet_v2_x1_0, shufflenet_v2_x1_5 |
| MobileNet    | mobilenet_v2, mobilenet_v3_large, mobilenet_v3_small         |
| MNASNet      | mnasnet0_5, mnasnet0_75, mnasnet1_0, mnasnet1_3              |

In [3]:
from pixelmed_calc.custom.components.comp2 import extract, init_from_model, init_from_onekey

model_name = 'resnet50'
#model, transformer, device = init_from_model(model_name=model_name, transfer_learning=False)
model, transformer, device = init_from_onekey(r'E:\function\note2-深度学习分类\ViT\viz')
for n, m in model.named_modules():
    print('Feature name:', n, "|| Module:", m)

[2023-10-24 19:43:43 - vit.py:  93]	INFO	正在使用 ViT模型，具体参数为：images_size=256, patch_size=32, num_classes=3, dim=1024, depth=6, heads=16, mlp_dim=2048


Feature name:  || Module: ViT(
  (to_patch_embedding): Sequential(
    (0): Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=32, p2=32)
    (1): LayerNorm((3072,), eps=1e-05, elementwise_affine=True)
    (2): Linear(in_features=3072, out_features=1024, bias=True)
    (3): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (dropout): Dropout(p=0.0, inplace=False)
  (transformer): Transformer(
    (norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (layers): ModuleList(
      (0): ModuleList(
        (0): Attention(
          (norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (attend): Softmax(dim=-1)
          (dropout): Dropout(p=0.0, inplace=False)
          (to_qkv): Linear(in_features=1024, out_features=3072, bias=False)
          (to_out): Sequential(
            (0): Linear(in_features=1024, out_features=1024, bias=True)
            (1): Dropout(p=0.0, inplace=False)
          )
        )
        (1): FeedForward(
          (net

## 可视化卷积层

`Feature name:` 之后的名称为要可视化的层，例如`layer4.2.conv3`, 一般深度学习特征提取最后一层卷积层

** 注意 ** : 可视化的层，一定为带有`conv`的卷积层，而且一般是最后一层。

In [6]:
target_layer = "to_latent"
gradcam = monai.visualize.GradCAM(nn_module=model, target_layers=target_layer)

## 打印可视化界面

In [7]:
from pixelmed_calc.datasets.image_loader import default_loader
from pixelmed_calc.custom.components.comp2 import show_cam_on_image
import torch

for sample in samples:
    img = default_loader(sample)
    sample_ = transformer(img)
    sample_  = sample_.view(1, *sample_.size()).to(device)
    res_cam = gradcam(x=sample_, class_idx=None)
    fig, axes = plt.subplots(1, 2, figsize=(20, 10), facecolor='white')
    axes[0].imshow(res_cam[0][0].cpu(), cmap='jet')
    axes[1].imshow(img.resize(sample_.size()[2:]))
    plt.show()
    
    plt.figure(figsize=(10, 10))
    plt.imshow(show_cam_on_image(img.resize(sample_.size()[2:]), res_cam[0][0].cpu(), use_rgb=True))

ValueError: Input and output must have the same number of spatial dimensions, but got input with spatial dimensions of [] and output size of torch.Size([224, 224]). Please provide input tensor in (N, C, d1, d2, ...,dK) format and output size in (o1, o2, ...,oK) format.